In [14]:
#importing langchain modules

from langchain.document_loaders import UnstructuredExcelLoader
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain import OpenAI, VectorDBQA
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.document_loaders import TextLoader
import tiktoken
from langchain.callbacks import get_openai_callback
#importing the os

from config import OPENAI_API_KEY


In [15]:
#loading the excel files
loader = UnstructuredExcelLoader(
    "excel/database.xlsx"
)

docs = loader.load()



In [16]:
text_splitter_character = RecursiveCharacterTextSplitter(
    chunk_size=10000, chunk_overlap=1000
)

 
#docs = TextLoader('file.txt').load()
#embedding
all_splits = text_splitter_character.split_documents(docs)
#db = FAISS.from_documents(text_character, OpenAIEmbeddings())
# print(type(all_splits))


In [17]:
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    token  = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(token)
token_counts = [tiktoken_len(doc.page_content) for doc in all_splits]
print(sum(token_counts))

221639


In [18]:
#Embed and store the texts
#Supplying a persist_directory will stoer the embeddings on disk
persist_directory = "db"

# create the open-source embedding function
vectordb = Chroma.from_documents(documents=all_splits,embedding=OpenAIEmbeddings(),
        persist_directory=persist_directory)

#persist the db to disk
vectordb.persist()
vectordb = None

vectordb = Chroma.from_documents(documents=all_splits,embedding=OpenAIEmbeddings(),
        persist_directory=persist_directory)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID fe0721ee6440808fa601035d9a739752 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID fe0721ee6440808fa601035d9a739752 in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID fe0721ee6440

In [19]:
retrieval = vectordb.as_retriever(search_kwargs={"k":2})
#docs = retrieval.get_relevant_documents("Who is Rucha Joshi")


In [22]:
with get_openai_callback() as cb:
        qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="map_rerank", retriever=retrieval,
                return_source_documents=True, verbose=True)
        print(qa_chain("Which professor should  meet to know more about DNA and why"))
total_tokens = cb.total_tokens
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)
print(total_tokens)



> Entering new  chain...

> Finished chain.
{'query': 'Which professor should  meet to know more about DNA and why', 'result': ' You should meet Dr. Dhiraj Sinha to know more about DNA because he is a faculty member.', 'source_documents': [Document(page_content='faculty\n      \n    \n    \n      Dr. Dhiraj Sinha', metadata={'source': 'excel/database.xlsx'}), Document(page_content='faculty\n      \n    \n    \n      Dr. Dhiraj Sinha', metadata={'source': 'excel/database.xlsx'})]}
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
120


In [23]:
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/index/ (stored 0%)
  adding: db/index/index_metadata_47f97897-52dc-4c38-a8a6-abc32c520d1b.pkl (deflated 11%)
  adding: db/index/id_to_uuid_47f97897-52dc-4c38-a8a6-abc32c520d1b.pkl (deflated 37%)
  adding: db/index/uuid_to_id_47f97897-52dc-4c38-a8a6-abc32c520d1b.pkl (deflated 41%)
  adding: db/index/index_47f97897-52dc-4c38-a8a6-abc32c520d1b.bin (deflated 17%)
  adding: db/chroma-embeddings.parquet (deflated 29%)
  adding: db/chroma-collections.parquet (deflated 50%)


In [27]:
#To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

#delete the directory

IndexError: list index out of range

In [28]:
!rm -rf db/

##Restarting the runtime

In [ ]:
!unzip db.zip

In [ ]:
#importing langchain modules

from langchain.document_loaders import UnstructuredExcelLoader
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain import OpenAI, VectorDBQA
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.document_loaders import TextLoader
import tiktoken
from langchain.callbacks import get_openai_callback
#importing the os

from config import OPENAI_API_KEY


In [ ]:
persist_directory = "db"

# create the open-source embedding function
vectordb2 = Chroma(embedding_function=OpenAIEmbeddings(),
        persist_directory=persist_directory)

In [ ]:
retrieval = vectordb2.as_retriever(search_kwargs={"k":2})

{'query': 'Which professor can I meet to learn C++ from and give information about him', 'result': ' You can meet Dr. Srikant Srinivasan to learn C++ from. He is a faculty member at the university.', 'source_documents': [Document(page_content='faculty\n      \n    \n    \n      Dr. Srikant Srinivasan', metadata={'source': 'excel/database.xlsx'}), Document(page_content='faculty\n      \n    \n    \n      Dr. Srikant Srinivasan', metadata={'source': 'excel/database.xlsx'}), Document(page_content='faculty\n      \n    \n    \n      Dr. Srikant Srinivasan', metadata={'source': 'excel/database.xlsx'}), Document(page_content='faculty\n      \n    \n    \n      Dr. Srikant Srinivasan', metadata={'source': 'excel/database.xlsx'})]}